# Het verband tussen stress en een groene leefomgeving

### Perspectief 1

Het hebben van een groene leefomgeving helpt bij het verminderen van stress. Mensen in steden met veel groen melden vaker lagere stressniveaus en een betere mentale gezondheid dankzij de beschikbaarheid van groen die ze hebben.

**Argument 1**  
In Figuur 3 is te zien dat over het algemeen bij een laag aandeel aan groen in een land het percentage mensen zonder stressklachten hoger is. Dit toont aan dat het belang van groen in een land bijdraagt aan het verminderen van stress.

**Argument 2**  
In Figuur 1 en Figuur 2 is te zien dat bij de landen die te vergelijken vallen de meeste landen die een hoog aandeel van groen hebben in de oppervlakte ook een laag stresspercentage hebben. Dit betekent dat de hoeveelheid groen per inwoner bijdraagt aan het creëren van een lager stresspercentage.

**Argument 3**
In Figuur 5 is te zien dat landen met meer vierkante meters groen zoals Zwitserland een veel lager stresspercentage hebben dan landen die weinig vierkante meters groen hebben. Zo valt op dat de kleinste paar landen allemaal een hoger stresspercentage hebben.

In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

oecd_df = pd.read_csv('original_oecd.csv')
#nutteloze kolommen verwijderen
kolommen_verwijderen = ['ACTION', 
                        'STRUCTURE', 
                        'STRUCTURE_ID', 
                        'STRUCTURE_NAME', 
                        'FREQ', 
                        'Frequency of observation', 
                        'MEASURE', 
                        'Measure', 
                        'UNIT_MEASURE', 
                        'Unit of measure', 
                        'PM_CONCENTRATION', 
                        'PM concentration level', 
                        'TIME_SEASON', 
                        'Time of the day and season', 
                        'TERRITORIAL_LEVEL', 
                        'Territorial level', 
                        'TIME_PERIOD', 
                        'Observation value', 
                        'OBS_STATUS', 
                        'UNIT_MULT', 
                        'Unit multiplier', 
                        'Time period', 
                        'Observation status', 
                        'DECIMALS', 
                        'Decimals']
oecd_df = oecd_df.drop(kolommen_verwijderen, axis=1)
# Edgecase: REF_AREA begint bij alle landen behalve Mexico met 2 letters
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^MEX', 'MX', regex=True)
# sommige verkeerde landcodes worden hier vervangen door degene die correspondeert met iso_df
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^UK', 'GB', regex=True)
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^KOR', 'KR', regex=True)
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^EL', 'GR', regex=True)
# kolom renamen voor duidelijkheid
oecd_df = oecd_df.rename(columns={'OBS_VALUE': 'Gemiddelde aandeel van groen in oppervlakte van stad'})

#Om de data per land te kunnen aggregeren, heb ik landsnamen als extra kolom Percentage mensen die Nee hebben geantwoorddig.
iso_df = pd.read_csv('23digit.csv')
iso_mapping = dict(zip(iso_df['2digit'], iso_df['Country']))

def get_country(geocode):
    country_code = ''.join(filter(str.isalpha, geocode))[:2]  # Verwijder alle niet-alfabetische tekens en neem de eerste twee letters
    return iso_mapping.get(country_code, 'Unknown')

oecd_df['Country'] = oecd_df['REF_AREA'].apply(get_country)
oecd_df = oecd_df.merge(iso_df, on='Country')
oecd_df = oecd_df.drop(['2digit', 'Numeric'], axis=1)

# nu gaan we de stress en iso df mergen met de country als key
stress_df = pd.read_csv('global-emotions-stress.csv')
def percentage_to_float(percentage):
    '''
    Zet een waarde met een % erachter om naar een float zodat het vergeleken kan worden
    '''
    return float(percentage.strip('%'))

stress_df['YES'] = stress_df['YES'].apply(percentage_to_float)
stress_df['NO'] = stress_df['NO'].apply(percentage_to_float)

stress_df = stress_df.drop(["DON'T KNOW/REFUSED"], axis=1)

stress_df = pd.merge(stress_df, iso_df, on='Country')
# stress df kolommen renamen
stress_df = stress_df.rename(columns={'YES': 'Percentage mensen die Ja hebben geantwoord'})
stress_df = stress_df.rename(columns={'NO': 'Percentage mensen die Nee hebben geantwoord'})

# nu gaan we de green df aggregeren

agg_df = oecd_df.groupby('Country')['Gemiddelde aandeel van groen in oppervlakte van stad'].mean().reset_index()
# tijd om de twee datasets te mergen
merged_df = pd.merge(agg_df, stress_df, on='Country')

# nutteloze kolommen weghalen
merged_df = merged_df.drop(['2digit', 'Numeric'], axis=1)

,Country,Gemiddelde aandeel van groen in oppervlakte van stad,Percentage mensen die Ja hebben geantwoord,Percentage mensen die Nee hebben geantwoord,3digit
0,Australia,38.431250,42.0,58.0,AUS
1,Austria,40.800000,32.0,68.0,AUT
2,Belgium,45.183333,39.0,60.0,BEL
3,Bulgaria,34.814286,23.0,74.0,BGR
4,Canada,46.457692,51.0,49.0,CAN
5,Chile,21.296154,37.0,62.0,CHL
6,Colombia,41.843396,47.0,53.0,COL
7,Croatia,43.633333,36.0,63.0,HRV
8,Cyprus,19.033333,55.0,45.0,CYP
9,Denmark,48.625000,25.0,75.0,DNK


In [8]:
fig_green = px.choropleth(oecd_df, locations='3digit', color='Gemiddelde aandeel van groen in oppervlakte van stad', hover_name='Gemiddelde aandeel van groen in oppervlakte van stad', projection='natural earth', title='Figuur 1: Aantal groen per land')
fig_green.update_layout(showlegend=False, width=900, height=500)

fig_green.add_annotation(
    xref='paper', yref='paper',
    x=0.7, y=-0.10,  # Adjust y to position the annotation at the bottom
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Deze wereldkaart laat de gemiddelde aandeel groen ten opzichte van de oppervlakte van een<br>' +
         'stad per land zien. Als je met de muis over een land heen beweegt, dan zie je de precieze percentage.',
    font=dict(size=13),  # Adjust font size if necessary
)

fig_green.show()

fig_stress = px.choropleth(stress_df, locations='3digit', color='Percentage mensen die Ja hebben geantwoord', hover_name='Percentage mensen die Ja hebben geantwoord', projection='natural earth', title='Figuur 2: Stress per land')
fig_stress.update_layout(showlegend=False, width=900, height=500)

fig_stress.add_annotation(
    xref='paper', yref='paper',
    x=0.65, y=-0.10,  # Adjust y to position the annotation at the bottom
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Deze wereldkaart laat de percentage mensen zien die Ja hebben geantwoord op de enquêtevraag. <br>' +
         'Als je met de muis over een land heen beweegt, dan zie je de precieze percentage.',
    font=dict(size=13),  # Adjust font size if necessary
)

fig_stress.show()

### Perspectief 2

Hoewel groene leefomgevingen bijdragen aan een gevoel van welzijn, zijn andere factoren zoals werkdruk, persoonlijke gewoontes en gezondheid vaak de voornaamste reden voor hoge stressniveaus. Het hebben van een groene leefomgeving helpt niet bij het verminderen van stress indien mensen bijvoorbeeld een hoge werkdruk blijven ervaren.

**Argument 1**  
In Figuur 3 is te zien dat er landen zoals Japan zijn waar het aandeel groen laag is, maar het aantal mensen zonder stressklachten hoog is. Dit toont aan dat het aandeel groen bijdraagt aan het welzijn, maar niet helpt bij het verminderen van stress. Dit is ook bij Mexico het geval.

**Argument 2**  
In Figuur 4 is te zien dat Griekenland en Canada beide een hoog aandeel groen in hun land hebben, maar alsnog een ontzettend hoog stressniveau hebben. Dit laat zien dat het aandeel groen in een land haast geen bijdrage heeft aan het verminderen van het stressniveau. Ook ondersteunen de meeste landen in (Figuur 4) dit argument.

**Argument 3**
In Figuur 5 is te zien dat het niet uitmaakt of een land een hoog aandeel groen heeft. Er is namelijk te zien dat het alsnog per land verdeelt is of het stresspercentage laag of hoog is.


In [9]:
trace_green = go.Bar(
    x=merged_df['Country'],
    y=merged_df['Gemiddelde aandeel van groen in oppervlakte van stad'],
    name='Groen aandeel van land',
    marker=dict(color='rgb(102,194,165)')
)

trace_no = go.Bar(
    x=merged_df['Country'],
    y=merged_df['Percentage mensen die Nee hebben geantwoord'],
    name='Percentage mensen zonder stress klachten',
    marker=dict(color='rgb(252,141,98)')
)

layout = go.Layout(
    title='Figuur 3: Groen aandeel en percentage mensen zonder stress per land',
    xaxis=go.layout.XAxis(
        title='Land',
        type='category'
    ),
    yaxis=go.layout.YAxis(
        title='Percentage',
    ),
    barmode='group',
    height=500  # Restore original height
)

fig = go.Figure(data=[trace_green, trace_no], layout=layout)

fig.add_annotation(
    xref='paper', yref='paper',
    x=0.5, y=-0.2,  # Adjust y to a lower value to fit within the frame
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Deze barplot laat per land de groene aandeel en percentage mensen zonder stressklachten zien. <br>' +
         'Als je met de muis over een land heen beweegt, dan zie je de precieze percentage en de kolom waar die bij hoort.',
    font=dict(size=10),  # Decrease font size if necessary
    bordercolor='black',
    borderwidth=1,
    borderpad=4,
    bgcolor='white',  # Background color for better readability
    opacity=0.8
)

fig.show()

In [10]:
fig4 = px.scatter(
    merged_df, 
    x='Gemiddelde aandeel van groen in oppervlakte van stad', 
    y='Percentage mensen die Ja hebben geantwoord', 
    size='Gemiddelde aandeel van groen in oppervlakte van stad', 
    color='Country', 
    title='Figuur 4: Mensen met stress vs. Groen Percentage per Stad',
    text='3digit'
)

x = merged_df['Gemiddelde aandeel van groen in oppervlakte van stad']
y = merged_df['Percentage mensen die Ja hebben geantwoord']
coefficients = np.polyfit(x, y, 1)
trendline = np.poly1d(coefficients)

# trendline values
x_trend = np.linspace(x.min(), x.max(), 100)
y_trend = trendline(x_trend)

# trendline trace
fig4.add_trace(
    go.Scatter(
        x=x_trend, 
        y=y_trend, 
        mode='lines', 
        line=dict(color='red'),
        name='Trendline'
    )
)

fig4.update_layout(showlegend=False, width=1000, height=600)

fig4.add_annotation(
    xref='paper', yref='paper',
    x=0.65, y=-0.10,  # Adjust y to position the annotation at the bottom
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Deze scatterplot laat de percentage mensen zien die Ja hebben geantwoord op de enquêtevraag, <br>' +
         'ten opzichte van de gemiddelde aandeel groen van de oppervlakte van een stad. <br>' +
         'Als je met de muis over een land heen beweegt, dan zie je de precieze percentages. <br>' +
         'In de plot is te zien dat de bevolking van landen met meer groen oppervlak minder stress ervaren. ',
    font=dict(size=13),  # Adjust font size if necessary
)

fig4.show()


In [11]:
fig5 = px.bar(merged_df, x='Country', y='Gemiddelde aandeel van groen in oppervlakte van stad', 
              color='Percentage mensen die Ja hebben geantwoord', barmode='group',
              title='Figuur 5: Stressniveau per Stad in verhouding tot Groen Percentage')
fig5.update_layout(yaxis={'title':'aantal vierkante meter'}, xaxis={'title':'Stad'})

fig5.add_annotation(
    xref='paper', yref='paper',
    x=0.7, y=-1.5,  # Adjust y to position the annotation at the bottom
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Deze staafdiagram laat per land de percentage mensen zien die Ja hebben geantwoord op de enquêtevraag, <br>' +
         'ten opzichte van de gemiddelde aandeel groen van de oppervlakte van een stad. <br>' +
         'Als je met de muis over een land heen beweegt, dan zie je de precieze percentages. <br>' +
         'In de plot is te zien dat de staven die minder lang zijn een meer gele kleur hebben. <br> ' +
         'Dit betekent dat de bevolking van landen met minder groen meer stress ervaart',
    font=dict(size=13),  # Adjust font size if necessary
)

fig5.show()

Figuur 6 laat de distributie van het gemiddelde aandeel van groen gebied in oppervlakte van een stad zien

Figuur 7 Laat de distributie van waardes van Ja-antwoorden zien.

In [12]:
# Boxplot data voor de eerste boxplot
boxplot_data1 = oecd_df[['Gemiddelde aandeel van groen in oppervlakte van stad']].copy()
boxplot_data1['Gemiddelde aandeel van groen in oppervlakte van stad'] = boxplot_data1['Gemiddelde aandeel van groen in oppervlakte van stad'] / 100
# Boxplot data voor de tweede boxplot
boxplot_data2 = stress_df[['Percentage mensen die Ja hebben geantwoord']].copy()
boxplot_data2['Percentage mensen die Ja hebben geantwoord'] = boxplot_data2['Percentage mensen die Ja hebben geantwoord'] / 100

# Creëer subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Figuur 6: Distributie van Gemiddelde aandeel van groen in oppervlakte van stad', 'Figuur 7: Distributie van YES values'),
    horizontal_spacing=0.1
)

# Voeg eerste boxplot toe aan de eerste subplot
fig6 = px.box(
    boxplot_data1,
    y='Gemiddelde aandeel van groen in oppervlakte van stad',
    labels={'Gemiddelde aandeel van groen in oppervlakte van stad': 'Green Area (%)'}
)
for trace in fig6['data']:
    fig.add_trace(trace, row=1, col=1)

# Voeg tweede boxplot toe aan de tweede subplot
fig7 = px.box(
    boxplot_data2,
    y='Percentage mensen die Ja hebben geantwoord'
)
for trace in fig7['data']:
    fig.add_trace(trace, row=1, col=2)

# Update layout van de gecombineerde figuur
fig.update_layout(
#    title_text='Figuur 6 en 7: Distributie van Gemiddelde aandeel van groen in oppervlakte van stad en waardes van Ja-antwoorden',
    width=1200,
    height=600
)

# Specifieke y-as titels voor elke subplot
fig.update_yaxes(title_text='Gemiddelde aandeel van groen in oppervlakte van stad', tickformat=".0%", row=1, col=1)
fig.update_yaxes(title_text='Percentage of people with stress complaints', tickformat=".0%", row=1, col=2)

# Update de positie van de subplot titels om overlapping te voorkomen
fig.update_layout(
    annotations=[
        dict(
            text='Figuur 6: Distributie van Gemiddelde aandeel van groen in oppervlakte van stad',
            x=0.22,
            xref='paper',
            y=1.15,
            yref='paper',
            showarrow=False,
            font=dict(size=14)
        ),
        dict(
            text='Figuur 7: Distributie van waardes van Ja-antwoorden',
            x=0.78,
            xref='paper',
            y=1.15,
            yref='paper',
            showarrow=False,
            font=dict(size=14)
        )
    ]
)

fig.add_annotation(
    xref='paper', yref='paper',
    x=0.5, y=-0.5,  # Adjust y to position the annotation at the bottom
    showarrow=False,
    xanchor='center',
    yanchor='top',
    align='center',
    text='Figuur 6 laat de distributie van het gemiddelde aandeel van groen gebied in oppervlakte van een stad zien. <br>' +
        'Figuur 7 Laat de distributie van waardes van Ja-antwoorden zien',
    font=dict(size=13),  # Adjust font size if necessary
)
fig.show()

## Conclusie

Dit onderzoek heeft aangetoond dat er zeker een relatie bestaat tussen het hebben van een groene leefomgeving en het stressniveau van individuen. De perspectieven laten zien dat het hebben van een groene leefomgeving een invloed heeft op het stressniveau van een individu. De argumenten bij het tweede perspectief suggereren wel dat het hebben van een groene leefomgeving helpt maar bij lange na niet genoeg is om de stressniveau's zodanig te verlagen dat het de gezondheid van een individu goed kan verhogen. Hiermee valt er dus te concluderen dat een groene leefomgeving veel voordelen biedt voor de mentale en fysieke gezondheid, maar geen op zichzelf staande oplossing is voor het verminderen van stress.

## Referenties

- Gallup, Inc. (2024, 25 juni). What Is the World’s Emotional Temperature? - Gallup. Gallup.com. [https://news.gallup.com/interactives/248240/global-emotions.aspx](https://news.gallup.com/interactives/248240/global-emotions.aspx)

- List of country codes by alpha-2, alpha-3 code (ISO 3166). (z.d.). [https://www.iban.com/country-codes](https://www.iban.com/country-codes)

- OECD. (z.d.). City statistics: Green areas. © OECD. [https://stats.oecd.org/Index.aspx?QueryId=119735](https://stats.oecd.org/Index.aspx?QueryId=119735)

- RIVM. (2022, 17 mei). Effecten van groen op fysieke en mentale gezondheid. Geraadpleegd op 28 juni 2024, van [https://www.rivm.nl/nieuws/effecten-van-groen-op-fysieke-en-mentale-gezondheid](https://www.rivm.nl/nieuws/effecten-van-groen-op-fysieke-en-mentale-gezondheid)
